<a href="https://colab.research.google.com/github/kidane12/-furry-octo-potato/blob/master/Corpora_and_Vector_Spaces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:

import os
import tempfile
TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

Folder "/tmp" will be used to save temporary dictionary and corpus.


In [0]:

from gensim import corpora

In [5]:

!pip install -U gensim

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.6.0)


In [0]:
documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",              
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

In [0]:
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]

from pprint import pprint



In [10]:
dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'deerwester.dict'))  # store the dictionary, for future reference
print(dictionary)

2018-11-11 22:04:22,198 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-11-11 22:04:22,200 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2018-11-11 22:04:22,202 : INFO : saving Dictionary object under /tmp/deerwester.dict, separately None
2018-11-11 22:04:22,203 : INFO : saved /tmp/deerwester.dict


Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [0]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())

In [12]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'deerwester.mm'), corpus)  # store to disk, for later use

2018-11-11 22:06:02,442 : INFO : storing corpus in Matrix Market format to /tmp/deerwester.mm
2018-11-11 22:06:02,445 : INFO : saving sparse matrix to /tmp/deerwester.mm
2018-11-11 22:06:02,447 : INFO : PROGRESS: saving document #0
2018-11-11 22:06:02,450 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2018-11-11 22:06:02,452 : INFO : saving MmCorpus index to /tmp/deerwester.mm.index


In [13]:
for c in corpus:
    print(c)

[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]
[(3, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1)]


In [0]:
from smart_open import smart_open
class MyCorpus(object):
    def __iter__(self):
        for line in smart_open('mycorpus.txt', 'rb'):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.lower().split())

In [29]:
corpus_memory_friendly = MyCorpus() # doesn't load the corpus into memory!
print(corpus_memory_friendly)

In [31]:
for vector in corpus_memory_friendly:  # load one vector into memory at a time
    print(vector)

[(0, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (7, 1)]
[(2, 1), (7, 1), (8, 1)]
[(1, 1), (5, 1)]
[(3, 1), (6, 1), (7, 1)]
[]
[(10, 1)]
[(9, 1), (11, 1)]
[(11, 1)]


In [0]:
from six import iteritems
from smart_open import smart_open

In [36]:
dictionary = corpora.Dictionary(line.lower().split() for line in smart_open('mycorpus.txt', 'rb'))

2018-11-11 22:42:04,564 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-11-11 22:42:04,567 : INFO : built Dictionary(53 unique tokens: ['=', '["human', 'abc', 'applications",', 'computer']...) from 9 documents (total 71 corpus positions)


In [37]:
# remove stop words and words that appear only once
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist 
            if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]

# remove stop words and words that appear only once
dictionary.filter_tokens(stop_ids + once_ids)
print(dictionary)

Dictionary(9 unique tokens: ['computer', 'interface', 'response', 'system', 'user']...)


In [38]:
# create a toy corpus of 2 documents, as a plain Python list
corpus = [[(1, 0.5)], []]  # make one document empty, for the heck of it

corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'corpus.mm'), corpus)

2018-11-11 22:44:39,536 : INFO : storing corpus in Matrix Market format to /tmp/corpus.mm
2018-11-11 22:44:39,538 : INFO : saving sparse matrix to /tmp/corpus.mm
2018-11-11 22:44:39,540 : INFO : PROGRESS: saving document #0
2018-11-11 22:44:39,541 : INFO : saved 2x2 matrix, density=25.000% (1/4)
2018-11-11 22:44:39,542 : INFO : saving MmCorpus index to /tmp/corpus.mm.index


In [39]:
corpora.SvmLightCorpus.serialize(os.path.join(TEMP_FOLDER, 'corpus.svmlight'), corpus)
corpora.BleiCorpus.serialize(os.path.join(TEMP_FOLDER, 'corpus.lda-c'), corpus)
corpora.LowCorpus.serialize(os.path.join(TEMP_FOLDER, 'corpus.low'), corpus)

2018-11-11 22:45:03,439 : INFO : converting corpus to SVMlight format: /tmp/corpus.svmlight
2018-11-11 22:45:03,441 : INFO : saving SvmLightCorpus index to /tmp/corpus.svmlight.index
2018-11-11 22:45:03,443 : INFO : no word id mapping provided; initializing from corpus
2018-11-11 22:45:03,445 : INFO : storing corpus in Blei's LDA-C format into /tmp/corpus.lda-c
2018-11-11 22:45:03,446 : INFO : saving vocabulary of 2 words to /tmp/corpus.lda-c.vocab
2018-11-11 22:45:03,448 : INFO : saving BleiCorpus index to /tmp/corpus.lda-c.index
2018-11-11 22:45:03,452 : INFO : no word id mapping provided; initializing from corpus
2018-11-11 22:45:03,455 : INFO : storing corpus in List-Of-Words format into /tmp/corpus.low
2018-11-11 22:45:03,459 : WARNING : List-of-words format can only save vectors with integer elements; 1 float entries were truncated to integer value
2018-11-11 22:45:03,460 : INFO : saving LowCorpus index to /tmp/corpus.low.index


In [40]:
corpus = corpora.MmCorpus(os.path.join(TEMP_FOLDER, 'corpus.mm'))

2018-11-11 22:45:28,127 : INFO : loaded corpus index from /tmp/corpus.mm.index
2018-11-11 22:45:28,129 : INFO : initializing cython corpus reader from /tmp/corpus.mm
2018-11-11 22:45:28,131 : INFO : accepted corpus with 2 documents, 2 features, 1 non-zero entries


In [41]:
print(corpus)


MmCorpus(2 documents, 2 features, 1 non-zero entries)


In [43]:
print(list(corpus))

[[(1, 0.5)], []]


In [0]:
import gensim
import numpy as np
numpy_matrix = np.random.randint(10, size=[5,2])
corpus = gensim.matutils.Dense2Corpus(numpy_matrix)
numpy_matrix_dense = gensim.matutils.corpus2dense(corpus, num_terms=10)

In [45]:
print(numpy_matrix_dense)

[[2. 7.]
 [1. 8.]
 [5. 2.]
 [3. 7.]
 [2. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [46]:
from gensim import corpora, models, similarities
if (os.path.exists("/tmp/deerwester.dict")):
  dictionary = corpora.Dictionary.load('/tmp/deerwester.dict')
  corpus = corpora.MmCorpus('/tmp/deerwester.mm')
  print("Used files generated from first tutorial")
else:
  print("Please run first tutorial to generate data set")

2018-11-11 22:49:55,428 : INFO : loading Dictionary object from /tmp/deerwester.dict
2018-11-11 22:49:55,431 : INFO : loaded /tmp/deerwester.dict
2018-11-11 22:49:55,433 : INFO : loaded corpus index from /tmp/deerwester.mm.index
2018-11-11 22:49:55,433 : INFO : initializing cython corpus reader from /tmp/deerwester.mm
2018-11-11 22:49:55,434 : INFO : accepted corpus with 9 documents, 12 features, 28 non-zero entries


Used files generated from first tutorial


In [48]:
print(corpus)

MmCorpus(9 documents, 12 features, 28 non-zero entries)


In [49]:
tfidf = models.TfidfModel(corpus)

2018-11-11 22:51:53,462 : INFO : collecting document frequencies
2018-11-11 22:51:53,464 : INFO : PROGRESS: processing document #0
2018-11-11 22:51:53,467 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


In [50]:
print(tfidf)

TfidfModel(num_docs=9, num_nnz=28)


In [52]:
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow]) # step 2 -- use the model to transform vectors

[(0, 0.7071067811865476), (1, 0.7071067811865476)]


In [53]:
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
  print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(4, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [54]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2) # initialize an LSI transformation
corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

2018-11-11 22:58:02,973 : INFO : using serial LSI version on this node
2018-11-11 22:58:02,975 : INFO : updating model with new documents
2018-11-11 22:58:02,978 : INFO : preparing a new chunk of documents
2018-11-11 22:58:02,989 : INFO : using 100 extra samples and 2 power iterations
2018-11-11 22:58:02,991 : INFO : 1st phase: constructing (12, 102) action matrix
2018-11-11 22:58:02,999 : INFO : orthonormalizing (12, 102) action matrix
2018-11-11 22:58:03,026 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2018-11-11 22:58:03,036 : INFO : computing the final decomposition
2018-11-11 22:58:03,038 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2018-11-11 22:58:03,045 : INFO : processed documents up to #9
2018-11-11 22:58:03,049 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2018-11-11 22:58:03,052 : INFO : topic #

In [55]:
lsi.print_topics(2)

2018-11-11 22:58:43,904 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2018-11-11 22:58:43,905 : INFO : topic #1(1.476): -0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"time" + -0.320*"response" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"


[(0,
  '0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  '-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"time" + -0.320*"response" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

In [56]:
for doc in corpus_lsi: # both bow->tfidf and tfidf->lsi transformations are actually executed here, on the fly
  print(doc)

[(0, 0.06600783396090637), (1, -0.5200703306361844)]
[(0, 0.19667592859142796), (1, -0.7609563167700043)]
[(0, 0.08992639972446795), (1, -0.7241860626752498)]
[(0, 0.07585847652178504), (1, -0.6320551586003417)]
[(0, 0.10150299184980344), (1, -0.5737308483002957)]
[(0, 0.7032108939378303), (1, 0.16115180214026092)]
[(0, 0.8774787673119822), (1, 0.1675890686465979)]
[(0, 0.9098624686818568), (1, 0.14086553628719403)]
[(0, 0.6165825350569277), (1, -0.05392907566389127)]


In [57]:
lsi.save('/tmp/model.lsi') # same for tfidf, lda, ...

2018-11-11 23:04:13,326 : INFO : saving Projection object under /tmp/model.lsi.projection, separately None
2018-11-11 23:04:13,329 : INFO : saved /tmp/model.lsi.projection
2018-11-11 23:04:13,331 : INFO : saving LsiModel object under /tmp/model.lsi, separately None
2018-11-11 23:04:13,333 : INFO : not storing attribute projection
2018-11-11 23:04:13,337 : INFO : not storing attribute dispatcher
2018-11-11 23:04:13,339 : INFO : saved /tmp/model.lsi


In [58]:
lsi = models.LsiModel.load('/tmp/model.lsi')

2018-11-11 23:04:26,735 : INFO : loading LsiModel object from /tmp/model.lsi
2018-11-11 23:04:26,739 : INFO : loading id2word recursively from /tmp/model.lsi.id2word.* with mmap=None
2018-11-11 23:04:26,741 : INFO : setting ignored attribute projection to None
2018-11-11 23:04:26,744 : INFO : setting ignored attribute dispatcher to None
2018-11-11 23:04:26,748 : INFO : loaded /tmp/model.lsi
2018-11-11 23:04:26,749 : INFO : loading LsiModel object from /tmp/model.lsi.projection
2018-11-11 23:04:26,751 : INFO : loaded /tmp/model.lsi.projection


In [59]:
from gensim import corpora, models, similarities
dictionary = corpora.Dictionary.load('/tmp/deerwester.dict')
corpus = corpora.MmCorpus('/tmp/deerwester.mm') # comes from the first tutorial, "From strings to vectors"
print(corpus)

2018-11-11 23:09:44,190 : INFO : loading Dictionary object from /tmp/deerwester.dict
2018-11-11 23:09:44,193 : INFO : loaded /tmp/deerwester.dict
2018-11-11 23:09:44,194 : INFO : loaded corpus index from /tmp/deerwester.mm.index
2018-11-11 23:09:44,195 : INFO : initializing cython corpus reader from /tmp/deerwester.mm
2018-11-11 23:09:44,196 : INFO : accepted corpus with 9 documents, 12 features, 28 non-zero entries


MmCorpus(9 documents, 12 features, 28 non-zero entries)


In [60]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

2018-11-11 23:10:54,292 : INFO : using serial LSI version on this node
2018-11-11 23:10:54,294 : INFO : updating model with new documents
2018-11-11 23:10:54,295 : INFO : preparing a new chunk of documents
2018-11-11 23:10:54,298 : INFO : using 100 extra samples and 2 power iterations
2018-11-11 23:10:54,300 : INFO : 1st phase: constructing (12, 102) action matrix
2018-11-11 23:10:54,304 : INFO : orthonormalizing (12, 102) action matrix
2018-11-11 23:10:54,307 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2018-11-11 23:10:54,310 : INFO : computing the final decomposition
2018-11-11 23:10:54,312 : INFO : keeping 2 factors (discarding 43.156% of energy spectrum)
2018-11-11 23:10:54,314 : INFO : processed documents up to #9
2018-11-11 23:10:54,316 : INFO : topic #0(3.341): 0.644*"system" + 0.404*"user" + 0.301*"eps" + 0.265*"time" + 0.265*"response" + 0.240*"computer" + 0.221*"human" + 0.206*"survey" + 0.198*"interface" + 0.036*"graph"
2018-11-11 23:10:54,318 : INFO : topic #1(2

In [0]:
doc = "Human computer interaction"

In [62]:
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow] # convert the query to LSI space
print(vec_lsi)

[(0, 0.4618210045327159), (1, -0.07002766527900015)]


In [63]:
index = similarities.MatrixSimilarity(lsi[corpus])

2018-11-11 23:14:02,277 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2018-11-11 23:14:02,280 : INFO : creating matrix with 9 documents and 2 features
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [64]:
index.save('/tmp/deerwester.index')
index = similarities.MatrixSimilarity.load('/tmp/deerwester.index')

2018-11-11 23:14:51,855 : INFO : saving MatrixSimilarity object under /tmp/deerwester.index, separately None
2018-11-11 23:14:51,857 : INFO : saved /tmp/deerwester.index
2018-11-11 23:14:51,859 : INFO : loading MatrixSimilarity object from /tmp/deerwester.index
2018-11-11 23:14:51,861 : INFO : loaded /tmp/deerwester.index


In [65]:
print(index)

MatrixSimilarity<9 docs, 2 features>


In [0]:
sims = index[vec_lsi] # perform a similarity query against the corpus

In [67]:
print(list(enumerate(sims)))

[(0, 0.998093), (1, 0.93748635), (2, 0.9984453), (3, 0.9865886), (4, 0.90755945), (5, -0.12416792), (6, -0.10639259), (7, -0.09879464), (8, 0.050041765)]


In [0]:
sims = sorted(enumerate(sims), key=lambda item: -item[1])

In [69]:
print(sims)

[(2, 0.9984453), (0, 0.998093), (3, 0.9865886), (1, 0.93748635), (4, 0.90755945), (8, 0.050041765), (7, -0.09879464), (6, -0.10639259), (5, -0.12416792)]
